# Requirements

In [1]:
import random as rnd
import pandas as pd
import numpy as np
import matplotlib as plt


# Data

In [2]:
capfile = "Capacity.csv"
dayslotfile = "DaysSlots.csv"
regsfile = "R_Data.csv"

#  importing roomsdf CAPACITY
global roomsdf
roomsdf = pd.read_csv(capfile, names=['RoomID', 'Capacity'], dtype=np.int64)
roomsdf


,RoomID,Capacity
0,1,20
1,2,28
2,3,28
3,4,28
4,5,28
5,6,28
6,7,28
7,8,20
8,9,28
9,10,28


In [3]:

#  importing DAY SLOTS
global slotsdf
slotsdf = pd.read_csv(dayslotfile, dtype=np.int64)
slotsdf


,Day,Slots
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2
5,6,2
6,7,2
7,8,2
8,9,2
9,10,2


In [34]:
#  importing REGISTRATION data
global regsdf
regsdf = pd.read_csv(regsfile)
regsdf = regsdf.groupby('SID\CID').sum()
regsdf = regsdf.astype(bool)
regsdf

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385],
           dtype='int64', name='SID\CID', length=1385)

In [5]:
#  STUDENTS
global studentsdf
studentsdf = regsdf.index.to_numpy()
studentsdf


array([   1,    2,    3, ..., 1383, 1384, 1385], dtype=int64)

In [6]:
#  COURSES
global coursesdf
coursesdf = regsdf.columns
coursesdf = np.array(coursesdf, np.int64)
coursesdf


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71], dtype=int64)

In [7]:
totalrooms = roomsdf['RoomID'].size
totalstudents = studentsdf.size
totalslots = slotsdf['Slots'].sum()
totalcourses = coursesdf.size


In [8]:
# Related to room

def get_room_cap(roomid):
    return roomsdf.loc[roomsdf['RoomID'] == roomid].iloc[0, 1]


def set_room_cap(roomid, capacity):
    roomsdf.loc[roomsdf['RoomID'] == roomid].iloc[0, 1] = capacity


def get_room(index):
    return roomsdf.iloc[index, 0]

# Related to coursesdf


def get_course_students(courseid):
    return regsdf.loc[regsdf[str(courseid)] == True].index


def get_student_courses(studentid):
    lst = regsdf.loc[studentid]
    lst = lst.to_frame()
    return lst[lst[studentid] == True].index.to_numpy(dtype=np.int64)


def student_taking_course(studentid, courseid) -> bool:
    return regsdf.loc[studentid][str(courseid)]


# Representation

## Gene
The chromosome is made up of many genes. In our program we are using course, students, room and slot as our gene. So When these many genes will combine will make a chromosome.

In other words, you can say this is our representation of the genes.

Gene=(course, students, room, slot)

In [9]:
class Gene:
    def __init__(self, course=None, room=None, slot=None, students=None,):
        # if there are None value then we will use random values
        if course is None:
            course = self.get_rand_course()
        if room is None:
            room = self.get_rand_room()
        if slot is None:
            slot = self.get_rand_slot()
        if students is None:
            students = self.get_rand_students(course, room)

        # copying to the self variables
        self.course = course
        self.room = room
        self.slot = slot
        self.students = students

    def __str__(self) -> str:
        return (
            "Gene[" +
            "course="+str(self.course)+", " +
            "room="+str(self.room)+", " +
            "slot="+str(self.slot)+", " +
            "students="+str(self.students) +
            "]"
        )

    def __repr__(self) -> str:
        return (
            "Gene[" +
            "course="+str(self.course)+", " +
            "room="+str(self.room)+", " +
            "slot="+str(self.slaot)+", " +
            "students="+str(len(self.students)) +
            "]"
        )

    def get_rand_course(self):
        return coursesdf[rnd.randint(0, totalcourses-1)]

    def get_rand_room(self):
        return get_room(rnd.randint(0, totalrooms-1))

    def get_rand_slot(self):
        return rnd.randint(1, totalslots)

    def get_rand_students(self, course, room):
        course_studs = get_course_students(course)
        return np.array([course_studs[rnd.randint(0, len(course_studs) - 1)] for _ in range(get_room_cap(room))])


## Chromosome
A array of genes. In our case a chromosome is whole representation of timetabl.

In [10]:
class Chromosome:
    def __init__(self, genes=None, fitnessval=None, rng=None):
        # filling random values if None
        if rng is None:
            rng = rnd.randint(totalcourses, totalcourses+30)
        if genes is None:
            genes = [Gene() for _ in range(0, rng)]
        if fitnessval is None:
            fitnessval = 0

        self.genes = np.array(genes)
        self.fitnessval = fitnessval
        self.detailfitness = {}

    # for genes list indexing
    def __getitem__(self, index):
        return self.genes[index]

        # for printing
    def __str__(self) -> str:
        return (
            "Chrom[" +
            "Genes="+str(len(self.genes))+", " +
            "Fitness="+str(self.fitnessval) +
            "]"
        )

    def __repr__(self) -> str:
        return self.__str__()

    # Courses of every gene
    def get_courses(self):
        return np.array([gene.course for gene in self.genes])

    def get_slots(self):
        return np.array([gene.slot for gene in self.genes])

    def get_rooms(self):
        return np.array([gene.room for gene in self.genes])

    def get_room_students(self, roomid):
        return np.array([gene.students for gene in self.genes if gene.room == roomid])

    def get_all_students(self):
        return np.concatenate([gene.students for gene in self.genes])


## Population
Population will contains many chromosome and we will apply the genetic algorithm on it

In [11]:
class Population:
    def __init__(self, chromosomes=None, rng=None) -> None:
        # filling random values if None
        if rng is None:
            rng = 10
        if chromosomes is None:
            chromosomes = [Chromosome() for _ in range(rng)]

        self.chromosomes = np.array(chromosomes)

    def __str__(self) -> str:
        return (
            "Population[" +
            "size="+str(len(self.chromosomes))+", " +
            "best=" +
            str(self.get_best())+", " +
            "Chromosomes=\n" +
            "\n".join([str(chrom) for chrom in self.chromosomes]) +
            "]"
        )

    def __repr__(self) -> str:
        return self.__str__()

    # for population indexing
    def __getitem__(self, index):
        return self.chromosomes[index]

    def get_best(self):
        if self.chromosomes.size == 0:
            return 0
        best_chromosome = self.chromosomes[0]
        for chromosome in self.chromosomes:
            if best_chromosome.fitnessval < chromosome.fitnessval:
                best_chromosome = chromosome
        return best_chromosome


# Fitness

## Constraints

### Hard Constraints

In [12]:
def one_exam_in_one_slot(chromosome):
    '''
    One course exam should be in a slot
    And after that slot or before that slot there should
    be no exam of the course
    More means bad
    '''

    courses = chromosome.get_courses()
    slots = chromosome.get_slots()

    courseslot = np.array([[course, slot]
                           for slot, course in zip(slots, courses)])

    # sorting
    courseslot = courseslot[courseslot[:, 0].argsort()]

    course_slots = np.split(courseslot[:, 1], np.unique(
        courseslot[:, 0], return_index=True)[1][1:])

    # now in course_slots. Course has list of slots in front of we
    # in our case it should be not longer than 1
    # counter the conflicts in which the slots are more than 1

    lst = [len(slots) for slots in course_slots]

    return sum(lst)-len(lst)


def one_room_have_one_exam(chromosome):
    '''
    One room should have one exam at a given time
    Count of the conflicts (more means bad)

    Will be checking the same_slot, same_room. Which means
    that at the given slot the room is beign used twice.
    We are not looking at the course because there should 
    be no duplicated in chromosome for the same room and same slot
    '''

    rooms = chromosome.get_rooms()
    slots = chromosome.get_slots()

    slotroom = [(slot, room) for slot, room in zip(slots, rooms)]

    # unique slot room and there counts
    _, counts = np.unique(slotroom, axis=0, return_counts=True)

    # print(counts)#debugging

    # counting of duplicate exams in one slot and one room
    dups = sum(counts)-len(counts)

    return dups


def student_one_exam_at_a_time(chromosome):
    '''
    At a given time, student can only give one exam

    In this I'll be also checking the duplication of student in the room
    and also in the course. Remeber the course can have multiple rooms within one slot.
    So in short I'll be looking for the same student in the same slot at multiple places

    Counting the conflics, more means bad
    '''

    dupstudents = 0  # duplicates of students in more than one slots
    dupexam = 0  # multiple exam of students at a time

    for i in range(len(chromosome.genes)):
        for student in chromosome.genes[i].students:
            for gene in chromosome:
                if gene == chromosome.genes[i] != gene.slot == chromosome.genes[i].slot:
                    if student in gene.students:  # multiple exam
                        dupexam += 1

        # counting duplicates of student in the same room
        dupstudents += len(chromosome.genes[i].students) - \
            len(set(chromosome.genes[i].students))

    return dupstudents+dupexam


def one_exam_per_course(chromosome):
    ''' 
    Every course should have one exam
    Not two Not zero, only one
    Count of the conflicts (more means bad)
    '''

    courses = chromosome.get_courses()

    uniquecourses, counts = np.unique(courses, return_counts=True)

    # counting the courses which don't have exam
    nocourseexam = totalcourses-len(uniquecourses)

    # counting of courses which have exam more than once
    dupcourseexam = sum(counts)-len(counts)

    return nocourseexam+dupcourseexam


def student_taking_correct_exam(chromosome):
    ''' 
    The Students must take every exam in which they are registered in
    Doesn't Count the number of missing courses for student XXX
    Count the number of missing student in courses
    more count means bad
    '''

    missing_students = 0  # number of students that are missing from exam

    for genes in chromosome:
        correct_sitting = 0
        for student in genes.students:
            stu_courses = get_student_courses(student)
            if genes.course in stu_courses:
                correct_sitting += 1
        missing_students += len(genes.students)-correct_sitting

    return missing_students


def room_cap_enough_for_students(chromosome):
    '''
    Every Gene hae room and student
    In here we will just check that there should be enough
    capacity to hold those students 

    Counting conflicts, more means bad
    '''

    # [(room capacity , number of students)]
    roomcap_students = [(get_room_cap(gene.room), len(gene.students))
                        for gene in chromosome]

    extra_stu = 0  # counting of extra students in room
    empty_space = 0  # counting of empaty space in room

    for cap, stu in roomcap_students:
        if stu > cap:
            extra_stu += stu-cap
        else:
            empty_space += cap-stu

    # extrastudents+(empty spaces)/10 beacause it's not good to have empty rooms
    return extra_stu+empty_space//10


In [13]:
HARD_CONSTRAINTS = [
    {
        # One course exam should be in a slot
        # And after that slot or before that slot there should
        # be no exam of the course

        "name": "One course exam should be in a slot",
        "function": one_exam_in_one_slot,
        "weight": 10,
        "fields": [
            "rooms"
        ]
    },
    {
        # rooms to course. The relation is 'n to 1'
        # A course can be in many rooms
        # But a room can only have one Course

        "name": "One room should have only one paper at a time",
        "function": one_room_have_one_exam,
        "weight": 10,
        "fields": [
            "rooms"
        ]
    },
    {
        # A student can't have more than one exam at a time

        "name": "One student should have one exam at a time",
        "function": student_one_exam_at_a_time,
        "weight": 1,
        "fields": [
            "students"
        ]
    },
    {
        # Every course should have exam

        "name": "Every Course should have Exam",
        "function": one_exam_per_course,
        "weight": 10,
        "fields": [
            "course"
        ]
    },
    {
        # Every student should have exam of there registered courses

        "name": "Every Student should have Exam",
        "function": student_taking_correct_exam,
        "weight": 1,
        "fields": [
            "course"
        ]
    },

    {
        "name": "Rooms should have enough space for the present Course Students",
        "function": room_cap_enough_for_students,
        "weight": 1,
        "fields": [
            "students"
            # "rooms" XXX because if we keep changing room and students are 10000 than we
            # will be stuck in infinit loop
        ]
    }
]


### Soft Constraints

## Fitness Calculation

In [14]:
def cal_fitness(chromosome):
    constraints_score = {}  # scores for the constraint pass
    mutate_fields = []  # field that requires mutation
    tscore = 0

    # Checking Hard Constraints
    for constraint in HARD_CONSTRAINTS:
        score = constraint['function'](
            chromosome)*constraint['weight']
        tscore += score

        constraints_score[constraint['name']] = score/constraint['weight']
        # constraints_score += score
        # if score > 10:
        #     # threshold setting
        #     # score > 10 means bad score
        #     if constraint["fields"] not in mutate_fields:
        #         mutate_fields += constraint["fields"]

    chromosome.detailfitness = constraints_score

    # Assigning the calculated fitness to the chromosome
    # range is 0-1. 0 beign the lowest and 1 means the perfect
    actualfitness = 1 / ((1.0*tscore+1))
    chromosome.fitnessval = actualfitness

    return actualfitness, mutate_fields


# Selection
# TODO

In [15]:
def roulette_wheel(population):
    pass


In [16]:
def elitism(population):
    pass


In [17]:
def select_best_parents(population):
    # Selects the best mom and dad
    return roulette_wheel(population) if rnd.randint(0, 1) else elitism(population)


# Crossover
# TODO

# Initialization

In [18]:
def init_population(rng=10):
    return Population(rng=rng)


In [19]:
newpop = init_population(10)
newpop


Population[size=10, best=Chrom[Genes=101, Fitness=0], Chromosomes=
Chrom[Genes=101, Fitness=0]
Chrom[Genes=95, Fitness=0]
Chrom[Genes=94, Fitness=0]
Chrom[Genes=87, Fitness=0]
Chrom[Genes=93, Fitness=0]
Chrom[Genes=77, Fitness=0]
Chrom[Genes=82, Fitness=0]
Chrom[Genes=71, Fitness=0]
Chrom[Genes=85, Fitness=0]
Chrom[Genes=96, Fitness=0]]

In [20]:
for chrom in newpop:
    cal_fitness(chrom)


In [21]:
newpop

Population[size=10, best=Chrom[Genes=71, Fitness=0.00030147723846849563], Chromosomes=
Chrom[Genes=101, Fitness=0.0002206531332744925]
Chrom[Genes=95, Fitness=0.0002295684113865932]
Chrom[Genes=94, Fitness=0.00023946360153256704]
Chrom[Genes=87, Fitness=0.00024067388688327315]
Chrom[Genes=93, Fitness=0.0002346316283435007]
Chrom[Genes=77, Fitness=0.0002744990392533626]
Chrom[Genes=82, Fitness=0.0002655337227827934]
Chrom[Genes=71, Fitness=0.00030147723846849563]
Chrom[Genes=85, Fitness=0.00026116479498563595]
Chrom[Genes=96, Fitness=0.00022619316896629722]]

In [22]:
newpop[1].detailfitness

{'One course exam should be in a slot': 42.0,
 'One room should have only one paper at a time': 3.0,
 'One student should have one exam at a time': 3305.0,
 'Every Course should have Exam': 60.0,
 'Every Student should have Exam': 0.0,
 'Rooms should have enough space for the present Course Students': 0.0}

In [23]:
best=newpop.get_best()
best

Chrom[Genes=71, Fitness=0.00030147723846849563]

In [24]:
best.detailfitness

{'One course exam should be in a slot': 25.0,
 'One room should have only one paper at a time': 4.0,
 'One student should have one exam at a time': 2526.0,
 'Every Course should have Exam': 50.0,
 'Every Student should have Exam': 0.0,
 'Rooms should have enough space for the present Course Students': 0.0}

# Representation of so far Best Timetable

In [35]:
def chromosome_to_df(chromosome):
    table = np.empty((totalrooms, totalslots), dtype=object)

    for course, room, slot in [[gene.course, gene.room, gene.slot] for gene in chromosome]:
        i = room-1
        j = slot-1
        table[i, j] = str(course) if table[i,
                                           j] is None else table[i, j]+", "+str(course)

    table = pd.DataFrame(
        table, columns=np.arange(1, totalslots+1).tolist(), index=np.arange(1, totalrooms+1).tolist())
    table.index.name = "RID/SID"
    return table


In [36]:
table = chromosome_to_df(best)
table.to_csv('best.csv')
table


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
RID/SID,,,,,,,,,,,,,,,,,,,,
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,30,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,36,None,None,None
7,None,None,None,None,28,None,None,None,None,None,None,None,None,None,None,None,None,None,64,None
8,None,None,None,41,15,None,None,None,None,None,None,None,None,None,48,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,35,None,None,None,None
